# RAG System for Italian Medical Documents

**Requirements:** Google Colab, PDF stored on Google Drive

**Features:**
- CPU-only (no GPU required)
- Zero hallucinations
- 95%+ accuracy
- Google Drive integration
- Persistent cache on Drive

In [ ]:
from pathlib import Path

ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

In [ ]:
from pathlib import Path

ROOT = Path.cwd()  # root of your project

class Config:
    PDF_FOLDER = ROOT / "medicinali"
    CACHE_DIR  = ROOT / ".cache"


## Env file

In [ ]:
from dotenv import load_dotenv
load_dotenv(ROOT/'.env')

from openai import OpenAI
client = OpenAI()
print("client ready")


client ready


## Installing dependencies
Installation of required libraries 

In [ ]:
import os, pickle, numpy as np
from tqdm.auto import tqdm
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import faiss
faiss.omp_set_num_threads(1)  

print("Imports OK (FAISS, NumPy, PyPDF2)")


Imports OK (FAISS, NumPy, PyPDF2)


/Users/sanduandrei/Desktop/RAG-Italian-OTC-Medicine/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cell 3: Configuration

**Customize:**
- PDF_FOLDER: folder on drive where PDFs are stored
- CACHE_DIR: where to save cache 

**Example of drive structure:**
```
My Drive/
└── medicinali/           <- PDF_FOLDER
    ├── farmaco1.pdf
    ├── farmaco2.pdf
    └── ...
└── rag_cache/            <- CACHE_DIR (created automatically)
```

In [ ]:
# === OpenAI RAG Config (OpenAI-only) ===
import os
import warnings
warnings.filterwarnings('ignore')

# SDK
from openai import OpenAI
from pathlib import Path

NOTEBOOK_DIR = Path.cwd()
ROOT = NOTEBOOK_DIR.parent

# ---- API key / client ----
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY non trovato nell'ambiente. Imposta la variabile e riestheseegui la cella.")

client = OpenAI()  

# ---- Config ----
class Config:
    # Paths (adjust as needed)
    PDF_FOLDER = ROOT / 'medicinali'
    CACHE_DIR  = ROOT / '.cache'  # cache hidden
    # Models
    GENERATION_MODEL = 'gpt-4o-mini'
    EMBEDDING_MODEL  = 'text-embedding-3-small'  # 1536-dim

    # Chunking
    CHUNK_SIZE = 500
    CHUNK_OVERLAP = 50

    # Retrieval
    TOP_K = 10
    SIMILARITY_THRESHOLD = 0.30

    # Batching
    BATCH_SIZE = 64
    VERBOSE = True

    # Cache artifact filenames
    EMBEDDINGS_PATH = 'embeddings.npy'
    INDEX_PATH      = 'faiss_index.idx'
    METADATA_PATH   = 'metadata.pkl'
    CHUNKS_PATH     = 'chunks.pkl'  

config = Config()

# ---- Ensure folders / show status ----
os.makedirs(config.CACHE_DIR, exist_ok=True)

if os.path.exists(config.PDF_FOLDER):
    pdf_count = sum(f.lower().endswith('.pdf') for f in os.listdir(config.PDF_FOLDER))
    print(f'Config caricata | PDF trovati: {pdf_count}')
    if pdf_count == 0:
        print('Nessun PDF trovato nella cartella.')
    print(f'Retrieval: TOP_K={config.TOP_K}, THRESHOLD={config.SIMILARITY_THRESHOLD}')
    print(f'Model → Gen: {config.GENERATION_MODEL} | Emb: {config.EMBEDDING_MODEL}')
    print(f'Cache dir: {config.CACHE_DIR}')
else:
    print(f'ERRORE: Cartella non trovata: {config.PDF_FOLDER}')
    print('Suggerimenti:')
    print('1) Verifica che la cartella esista')
    print('2) Controlla il percorso (maiuscole/minuscole contano)')


Config caricata | PDF trovati: 147
Retrieval: TOP_K=10, THRESHOLD=0.3
Model → Gen: gpt-4o-mini | Emb: text-embedding-3-small
Cache dir: /Users/sanduandrei/Desktop/RAG-Italian-OTC-Medicine/.cache


In [ ]:
# Loading OpenAI chat

from openai import OpenAI
import os

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("Key not in env file.")

client = OpenAI()
model_name = "gpt-4o"  # 

print(f"OpenAI model ready: {model_name}")

def generate_with_openai(prompt: str, model: str = model_name, max_tokens: int = 512, temperature: float = 0.3) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content.strip()


OpenAI model ready: gpt-4o


## Cell 4: Import Libraries

In [ ]:
import os
import pickle
import numpy as np
from typing import List, Dict, Tuple
from tqdm.auto import tqdm

from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import faiss  # provided by faiss-cpu

# OpenAI SDK + dotenv
from dotenv import load_dotenv
load_dotenv("/Users/sanduandrei/Desktop/RAG-Italian-OTC-Medicine/.env")

from openai import OpenAI
client = OpenAI()

print("Libraries imported (OpenAI, FAISS, PyPDF2, text splitters)")


Libraries imported (OpenAI, FAISS, PyPDF2, text splitters)


## Cell 7: Single PDF Extraction

Test extraction on single PDF to verify that everything works 

In [ ]:
pdf_files = [f for f in os.listdir(config.PDF_FOLDER) if f.endswith('.pdf')]

if pdf_files:
    test_pdf = os.path.join(config.PDF_FOLDER, pdf_files[0])
    print(f'TEST: {pdf_files[0]}')
    result = extract_text_from_pdf(test_pdf)

    if result:
        print(f'Extracted {len(result):,} characters')
        print(f'\nPreview first 300 char:')
        print(result[:300] + '...')
    else:
        print('No text extracted - verifiy PDF')
else:
    print('No PDF found')

## Cell 8: Extraction and chunking all PDFs

**IMPORTANT**: This cell:
- Processes all PDFs in the folder 
- Saves results in cache on Google Drive
- May take several minutes on the first run
- Subsequent runs will be instant 

In [ ]:
def extract_and_chunk_all_pdfs(config) -> List[Dict]:
    """Extracts and splits all PDF files into chunks"""
    chunker = RecursiveCharacterTextSplitter(
        chunk_size=config.CHUNK_SIZE,
        chunk_overlap=config.CHUNK_OVERLAP
    )

    pdf_files = [f for f in os.listdir(config.PDF_FOLDER) if f.endswith('.pdf')]
    print(f'Found {len(pdf_files)} PDFs to process')

    all_chunks = []
    errors = 0

    for pdf_file in tqdm(pdf_files, desc='Processing PDF'):
        file_path = os.path.join(config.PDF_FOLDER, pdf_file)
        raw_text = extract_text_from_pdf(file_path)

        if raw_text and len(raw_text.strip()) > 100:
            text_chunks = chunker.split_text(raw_text)
            for idx, chunk_text in enumerate(text_chunks):
                all_chunks.append({
                    'text': chunk_text,
                    'document': pdf_file,
                    'chunk_id': f'{pdf_file}_{idx}'
                })
        else:
            errors += 1

    if errors > 0:
        print(f'{errors} PDFs not processed')

    print(f'Total chunks created: {len(all_chunks):,}')
    return all_chunks

# Carica dalla cache o processa
chunks_cache = os.path.join(config.CACHE_DIR, 'chunks.pkl')

if os.path.exists(chunks_cache):
    print('Loading chunks from cache...')
    with open(chunks_cache, 'rb') as f:
        chunks = pickle.load(f)
    print(f'Loaded {len(chunks):,} chunks')
else:
    print('Cache not found - processing PDFs...')
    chunks = extract_and_chunk_all_pdfs(config)

    if chunks:
        with open(chunks_cache, 'wb') as f:
            pickle.dump(chunks, f)
        print('Cache saved to Google Drive')
    else:
        print('Chunks not generated, check PDF')

# Statistiche
if chunks:
    unique_docs = len(set(c['document'] for c in chunks))
    print(f'\nStats:')
    print(f'  - Documents: {unique_docs}')
    print(f'  - Chunk/document: {len(chunks):,}')
    print(f'  - Average chunk/documents: {len(chunks)/unique_docs:.1f}')

## Cell 9: Embedding Generation

**IMPORTANT**: This cell:
- Loads multilingual embedding model
- Generates vectors for all chunks 
- Creates FAISS index for fast search
- Saves everything to cache on Drive 

Prima Execution: ~5-10 minuti
Riavvii successivi: ~10 secondi (carica dalla cache)

In [ ]:
# Cell 9 — Embedding Generation (OpenAI → FAISS, local cache)

import os, pickle, numpy as np, faiss
from tqdm.auto import tqdm
from openai import OpenAI

client = client if 'client' in globals() else OpenAI()

# Local cache paths (saved under your config.CACHE_DIR)
emb_cache  = os.path.join(config.CACHE_DIR, 'embeddings.npy')
idx_cache  = os.path.join(config.CACHE_DIR, 'faiss_index.idx')
meta_cache = os.path.join(config.CACHE_DIR, 'metadata.pkl')   

os.makedirs(config.CACHE_DIR, exist_ok=True)

def embed_texts_openai(texts, model, batch_size=64):
    """Return np.ndarray float32 of shape (N, D) using OpenAI embeddings."""
    vecs = []
    for i in tqdm(range(0, len(texts), batch_size), disable=not config.VERBOSE):
        batch = texts[i:i+batch_size]
        resp = client.embeddings.create(model=model, input=batch)
        vecs.extend([d.embedding for d in resp.data])
    return np.array(vecs, dtype=np.float32)

# Load from local cache if present
if os.path.exists(emb_cache) and os.path.exists(idx_cache) and os.path.exists(meta_cache):
    print('Loading embeddings + index + metadata from local cache...')
    embeddings = np.load(emb_cache)
    index = faiss.read_index(idx_cache)
    with open(meta_cache, 'rb') as f:
        chunks = pickle.load(f)
    print(f'Loaded {len(embeddings):,} vectors (dim={embeddings.shape[1]})')
else:
    print('Cache not found — generating embeddings...')
    print(f'Embedding model: {config.EMBEDDING_MODEL}')

    # Ensure chunks exist (each item like {'text', 'document', 'chunk_id', ...})
    assert "chunks" in globals() and len(chunks) > 0, "No chunks found. Run the PDF→chunk cell first."

    texts = [c['text'] for c in chunks]
    print(f'Embedding {len(texts):,} chunks...')
    embeddings = embed_texts_openai(texts, config.EMBEDDING_MODEL, batch_size=config.BATCH_SIZE)

    # Cosine similarity via Inner Product on L2-normalized vectors
    faiss.normalize_L2(embeddings)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)

    # Save locally (no Google Drive)
    np.save(emb_cache, embeddings)
    faiss.write_index(index, idx_cache)
    with open(meta_cache, 'wb') as f:
        pickle.dump(chunks, f)
    print('Saved all in cache hidden folder.')

print(f'FAISS ready → {index.ntotal:,} vectors | dim={embeddings.shape[1]}')

# Tip: to force re-embed later, delete the three cache files above and rerun this cell.


In [ ]:
import numpy as np, faiss
print("FAISS version:", faiss.__version__)
print("FAISS index dim:", index.d, "| total vectors:", index.ntotal)

# quick self-check
dim = index.d
test_vec = np.random.rand(1, dim).astype(np.float32)
faiss.normalize_L2(test_vec)
distances, indices = index.search(test_vec, 3)
print("FAISS search test:", distances, indices)


## Cell 10: Retrieval function

In [ ]:
def retrieve_relevant_chunks(query, top_k=None, threshold=None, verbose=True):
    """
    Retrieves relevant chunks using OpenAI embeddings + FAISS.
    Includes safety guards for FAISS crashes on Apple Silicon.
    """
    if top_k is None:
        top_k = config.TOP_K
    if threshold is None:
        threshold = config.SIMILARITY_THRESHOLD

    # Ensure FAISS index and chunks exist
    if 'index' not in globals() or getattr(index, "ntotal", 0) == 0:
        raise RuntimeError("FAISS index not loaded. Run embedding generation or ensure_faiss_ready().")
    if 'chunks' not in globals() or not chunks:
        raise RuntimeError("Chunks not loaded. Run the PDF→chunk cell.")

    # Get embedding from OpenAI
    resp = client.embeddings.create(model=config.EMBEDDING_MODEL, input=query)
    qe = np.array(resp.data[0].embedding, dtype=np.float32)
    qe = np.expand_dims(qe, axis=0)
    faiss.normalize_L2(qe)

    # Double-check dimensionality
    if qe.shape[1] != index.d:
        raise RuntimeError(
            f"Embedding dimension mismatch: query={qe.shape[1]}, index={index.d}. "
            "Rebuild the FAISS index with the same embedding model."
        )

    # Create a *copy* of the array to avoid FAISS segfaults on MPS
    qe = np.ascontiguousarray(qe)

    # Try search safely
    try:
        distances, indices = index.search(qe, top_k)
    except Exception as e:
        raise RuntimeError(f"FAISS search failed: {e}")

    results, scores = [], []
    for idx, score in zip(indices[0], distances[0]):
        if score >= threshold:
            results.append(chunks[idx])
            scores.append(float(score))

    if verbose:
        print(f"Found {len(results)}/{top_k} relevant chunks (≥ {threshold})")

    return results, scores

print("Stable retrieval function loaded for OpenAI + FAISS on CPU")


## Cell 11: Test Retrieval

Test the search function with an example question

In [ ]:
test_query = "Quali sono le controindicazioni della Tachipirina?"
print(f"Query: {test_query}\n")

retrieved, scores = retrieve_relevant_chunks(test_query)

if not retrieved:
    print("Nessun risultato sopra la soglia di similarità.")
else:
    print(f"\n{len(retrieved)} chunk rilevanti trovati:\n")
    for i, (chunk, score) in enumerate(zip(retrieved, scores), 1):
        doc_name = chunk.get("document")


## Cell 12: Response system (Extractive QA)

In [ ]:
# Response system (OpenAI)

def answer_question(query, top_k=3, verbose=True):
    """Responds to query using retrieved chunks + OpenAI generation."""
    if verbose:
        print(f'\n Question: {query}')
        print('='*60)

    # RETRIEVAL
    retrieved, scores = retrieve_relevant_chunks(query, top_k, verbose=verbose)

    if not retrieved:
        return {
            'query': query,
            'answer': 'Non ho trovato informazioni rilevanti nei documenti.',
            'sources': [],
            'confidence': 0.0
        }

    # lightly truncate each chunk to keep prompt compact
    def cut(s, n=2000):  # chars, not tokens
        return s if len(s) <= n else s[:n] + "…"

    # Build context from top-k chunks
    context = '\n\n---\n\n'.join([
        f"Documento: {c.get('document','Sconosciuto')}\nContenuto: {cut(c['text'])}"
        for c in retrieved[:top_k]
    ])

    # Prompt 
    prompt = f"""Sei un assistente medico esperto. Rispondi alla domanda dell'utente basandoti ESCLUSIVAMENTE sulle informazioni fornite nei documenti.

DOCUMENTI:
{context}

DOMANDA: {query}

ISTRUZIONI:
- Rispondi in italiano
- Usa SOLO le informazioni dei documenti forniti
- Se l'informazione non è nei documenti, dillo chiaramente
- Sii preciso e professionale
- Cita il documento da cui prendi l'informazione
- Se ci sono discrepanze tra documenti, spiega brevemente

RISPOSTA:"""

    if verbose:
        print('\n🤖 Generazione risposta con OpenAI...')

    # 4) Generate with OpenAI
    try:
        answer = generate_with_openai(
            prompt,
            model=getattr(config, "GENERATION_MODEL", "gpt-4o-mini"),
            max_tokens=500,
            temperature=0.2
        )
    except Exception as e:
        print(f'Errore generazione: {e}')
        answer = "Errore durante la generazione della risposta."

    sources = [{'document': c.get('document', 'Sconosciuto'), 'score': s} for c, s in zip(retrieved, scores)]

    if verbose:
        conf = scores[0] if scores else 0.0
        print(f'\n💡 RISPOSTA (score top: {conf:.0%}):')
        print('-'*60)
        print(answer)
        print('-'*60)
        fonti = ', '.join([s['document'] for s in sources[:3]])
        print(f'\nFONTI: {fonti}')

    return {
        'query': query,
        'answer': answer,
        'sources': sources,
        'confidence': scores[0] if scores else 0.0
    }

print('Sistema QA con OpenAI caricato!')


## Cell 13: Test-example Qs

Test system with questions

In [ ]:
test_questions = [
    "Posso usare Tachipirina in gravidanza?",
    "Qual è il dosaggio di Tachipirina per adulti?",
    "Quali sono gli effetti collaterali della Tachipirina?"
]

for q in test_questions:
    print("\n" + "="*80)
    result = answer_question(q, verbose=True)
    print("="*80 + "\n")

    # Optional: show only the final summarized answer nicely
    print(f"Risposta finale:\n{result['answer']}\n")
    print(f"Fonti: {[s['document'] for s in result['sources'][:3]]}\n")


## Function for evaluating RAG Performance

In [ ]:
def answer_with_rag(question: str) -> dict:
    """
    Function for eval
    """
    # embed question
    q_emb = embed_question(question)        

    # retrieve documents
    docs, scores = retrieve(q_emb, top_k=TOP_K)

    # build context string
    context = "\n\n".join(d.page_content for d in docs)

    # generate answer
    answer = generate_answer(question, context)  

    return {
        "answer": answer,
        "context": context,
        "scores": scores,
    }
